In [1]:
import os
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import PowerTransformer

from imblearn.over_sampling import SMOTE

## models
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn.svm import LinearSVC

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, cross_val_predict, cross_validate

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, make_scorer, roc_auc_score

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

## to make it possible to display multiple output inside one cell 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.options.display.max_rows = 50
pd.set_option('display.float_format', lambda x: '%9.8f' % x)

In [2]:
## load data, split Xy, upsample with SMOTE, split train/test
data = pd.read_csv('data/Bar_data_encoded.csv')

## define X and y 
X = data.drop('Y', axis=1).reset_index(drop=True)
y = data.Y

## Handle imbalance
smote = SMOTE()

orig_vc = y.value_counts()
print(f"Orig: {100* orig_vc[1]/(orig_vc.sum()):.2f}% Yes")

X_sm, y_sm = smote.fit_resample(X, y)
sm_vc = y_sm.value_counts()
print(f"After SMOTE: {100* sm_vc[1]/(sm_vc.sum()):.2f}% Yes")

## Data splitting train/test
X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm, test_size=0.20, random_state=42)

Orig: 41.00% Yes
After SMOTE: 50.00% Yes


In [3]:
X_train.shape

(1904, 55)

## Testing RandomForestClassifier with Grid Search Cross-validation


In [4]:
roc_auc_scorer = make_scorer(roc_auc_score, average='weighted')

In [5]:
cols_to_view = ['rank_test_score', 'param_max_depth', 'param_max_features', 'param_min_samples_split', 
                'param_min_samples_leaf', 'param_n_estimators', 
                'mean_test_score', 'std_test_score', 'mean_train_score', 'std_train_score']

In [6]:
param_grid={'max_depth': [None, 5, 10, 15],
            'max_features': [5, 6, 7, 8, 9, 10],
            'min_samples_split': [2, 3, 4],
            'min_samples_leaf': [1, 2, 3, 4, 5],
            'n_estimators': [100, 150, 200]}

clf = RandomForestClassifier(oob_score=True, random_state=42)

grid_search1 = GridSearchCV(clf, param_grid, scoring=roc_auc_scorer, cv=5, return_train_score=True, n_jobs=4)
grid_search1.fit(X_train.values, y_train.values)

GridSearchCV(cv=5,
             estimator=RandomForestClassifier(oob_score=True, random_state=42),
             n_jobs=4,
             param_grid={'max_depth': [None, 5, 10, 15],
                         'max_features': [5, 6, 7, 8, 9, 10],
                         'min_samples_leaf': [1, 2, 3, 4, 5],
                         'min_samples_split': [2, 3, 4],
                         'n_estimators': [100, 150, 200]},
             return_train_score=True,
             scoring=make_scorer(roc_auc_score, average=weighted))

In [7]:
cv_results1 = pd.DataFrame(grid_search1.cv_results_)
cv_results1.loc[:, cols_to_view].sort_values('rank_test_score').head(10)

,rank_test_score,param_max_depth,param_max_features,param_min_samples_split,param_min_samples_leaf,param_n_estimators,mean_test_score,std_test_score,mean_train_score,std_train_score
95,1,None,7,3,1,200,0.81559421,0.01474375,0.99869108,0.00058729
953,2,15,8,4,1,200,0.81493292,0.01201869,0.98066847,0.00310739
98,3,None,7,4,1,200,0.81449757,0.02133878,0.99160151,0.00151958
137,4,None,8,2,1,200,0.81448388,0.01945913,1.00000000,0.00000000
231,5,None,10,4,1,100,0.81446999,0.01419823,0.99174275,0.00153543
995,6,15,9,3,1,200,0.81443962,0.01606664,0.99209921,0.00224116
46,7,None,6,2,1,150,0.81399873,0.01729692,1.00000000,0.00000000
904,8,15,7,3,1,150,0.81398222,0.01287788,0.98934789,0.00201093
818,9,15,5,4,1,200,0.81397401,0.01452180,0.97619089,0.00280864
97,10,None,7,4,1,150,0.81396020,0.02428817,0.99173948,0.00135171


In [8]:
grid_search1.best_estimator_

RandomForestClassifier(max_features=7, min_samples_split=3, n_estimators=200,
                       oob_score=True, random_state=42)

In [9]:
best_model = grid_search1.best_estimator_
best_model.oob_score_

0.819327731092437

In [10]:
cross_val_scores = cross_val_score(best_model, X_train, y_train, scoring=roc_auc_scorer, cv=5)
print(f"(weighted avg) AUC: {np.mean(cross_val_scores)} {np.std(cross_val_scores)}")

(weighted avg) AUC: 0.8155942079420482 0.01474374561469016


In [12]:
## fit best model on full train and test on test data
best_model = best_model.fit(X_train, y_train)

## use trained model to get predicted target var
# y_train_pred = best_model.predict(X_train)   
y_test_pred = best_model.predict(X_test) 

# print(f"(weighted avg) AUC: train {roc_auc_score(y_train, y_train_pred, average='weighted')}")
print(f"(weighted avg) AUC: TEST {roc_auc_score(y_test, y_test_pred, average='weighted')}")

(weighted avg) AUC: TEST 0.8046680442960844


In [11]:
featimp_df = pd.DataFrame({'feature_name': X.columns, 'importance': best_model.feature_importances_})
featimp_df.sort_values('importance', ascending=False).head(10)

,feature_name,importance
13,Bar,0.15864592
8,age,0.06437620
10,income,0.06067552
14,CoffeeHouse,0.04884793
9,education,0.04726093
15,CarryAway,0.04530429
17,Restaurant20To50,0.04444653
16,RestaurantLessThan20,0.04177953
11,temperature,0.03772857
7,time,0.03732591


# Attempting to look at the resulting decision tree nodes